In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1589,tt0346156,70000000,57958696,Sky Captain and the World of Tomorrow,Jude Law|Gwyneth Paltrow|Giovanni Ribisi|Angel...,Kerry Conran,Who will save us?,"When gigantic robots attack New York City, ""Sk...",107,Mystery|Action|Thriller|Science Fiction|Adventure,Paramount Pictures|Natural Nylon Entertainment...,9/17/2004,5.7,2004
1205,tt1392214,46000000,122126687,Prisoners,Hugh Jackman|Jake Gyllenhaal|Paul Dano|Maria B...,Denis Villeneuve,Every moment matters.,When Keller Dover's daughter and her friend go...,153,Drama|Thriller|Crime,Alcon Entertainment|Madhouse Entertainment|8:3...,9/18/2013,7.8,2013
958,tt0180052,100000000,7103973,The Adventures of Pluto Nash,Eddie Murphy|Randy Quaid|Rosario Dawson|Joe Pa...,Ron Underwood,Action's future has arrived...,"The year is 2087, the setting is the moon. Edd...",95,Action|Comedy|Science Fiction,Village Roadshow Pictures|Castle Rock Entertai...,8/15/2002,4.6,2002
1878,tt0176783,14000000,5108820,Gossip,Joshua Jackson|James Marsden|Kate Hudson|Lena ...,Davis Guggenheim,"It can turn you on, or turn on you.","On a beautiful college campus, something ugly ...",90,Drama|Mystery|Thriller,Warner Bros. Pictures,4/21/2000,5.3,2000
1572,tt0338526,160000000,300257475,Van Helsing,Hugh Jackman|Kate Beckinsale|Richard Roxburgh|...,Stephen Sommers,The One Name They All Fear.,Famed monster slayer Gabriel Van Helsing is di...,132,Horror|Adventure|Action,Universal Pictures|Stillking Films|Sommers Com...,5/5/2004,5.9,2004


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
# добавим колонку 'profit' - разница между кассовыми сборами и бюджетом
data['profit'] = data.revenue - data.budget
# Выполним разбиение данных в ячейках, где представлена информация через "|"
data.cast = data.cast.apply(lambda s: s.split('|'))
data.director = data.director.apply(lambda s: s.split('|'))
data.genres = data.genres.apply(lambda s: s.split('|'))
data.production_companies = data.production_companies.apply(
    lambda s: s.split('|'))

In [6]:
# Разобьем датафрейм - данные из списка станут отдельными строками
data = data.explode('cast')
data = data.explode('director')
data = data.explode('genres')
data = data.explode('production_companies')

In [7]:
# Создадим функцию, считающую количество слов. Прим: слова разделены ' '
def count_words(string):
    n = string.count(' ')
    return n+1

In [8]:
# Добавим колонки необходимые для анализа данных:
# Количество слов в 'описании'
data['count_word'] = data.overview.apply(count_words)
# Количество символов в 'названии'
data['length_title'] = data.original_title.apply(lambda s: len(s))
# Трансфомируем дату выхода в удобный для использования вид
data['release_date'] = pd.to_datetime(data.release_date)
# Месяц выходв фильма
data['release_month'] = data.release_date.apply(lambda data: data.month)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89552 entries, 0 to 1888
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               89552 non-null  object        
 1   budget                89552 non-null  int64         
 2   revenue               89552 non-null  int64         
 3   original_title        89552 non-null  object        
 4   cast                  89552 non-null  object        
 5   director              89552 non-null  object        
 6   tagline               89552 non-null  object        
 7   overview              89552 non-null  object        
 8   runtime               89552 non-null  int64         
 9   genres                89552 non-null  object        
 10  production_companies  89552 non-null  object        
 11  release_date          89552 non-null  datetime64[ns]
 12  vote_average          89552 non-null  float64       
 13  release_year     

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [10]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [11]:
# Создадим дафафрейм, который будем использовать для анализа данных,
# поскольку нам нет смысла использовать исходный датафрейм со всеми данными
data_find = pd.DataFrame()

In [12]:
# тут пишем ваш код для решения данного вопроса:
data_find = data[['imdb_id', 'original_title', 'budget']].drop_duplicates()
data_find[data_find.budget == data_find.budget.max()]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [13]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [14]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [15]:
# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать

In [16]:
data_find = data[['imdb_id', 'original_title', 'runtime']].drop_duplicates()
data_find[data_find.runtime == data_find.runtime.max()]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [17]:
answers['2'] = 'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [18]:
data_find[data_find.runtime == data_find.runtime.min()]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [19]:
answers['3'] = 'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [20]:
round(data_find.runtime.mean(), 0)

110.0

In [21]:
answers['4'] = '110'

# 5. Каково медианное значение длительности фильмов? 

In [22]:
data_find.runtime.median()

107.0

In [23]:
answers['5'] = '107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [24]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data_find = data[['imdb_id', 'original_title', 'profit']].drop_duplicates()
data_find[data_find.profit == data_find.profit.max()]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [25]:
answers['6'] = 'Avatar'

# 7. Какой фильм самый убыточный? 

In [26]:
data_find[data_find.profit == data_find.profit.min()]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [27]:
answers['7'] = 'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [28]:
data_find[data_find.profit > 0].profit.count()

1478

In [29]:
answers['8'] = '1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [30]:
data_find = data[['imdb_id', 'original_title',
                  'profit', 'release_year']].drop_duplicates()
data_find = data_find.query('release_year == 2008')

In [31]:
data_find[data_find.profit == data_find.profit.max()]

,imdb_id,original_title,profit,release_year
599,tt0468569,The Dark Knight,816921825,2008


In [32]:
answers['9'] = 'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [33]:
data_find = data[['imdb_id', 'original_title',
                  'profit', 'release_year']].drop_duplicates()
data_find = data_find.query('2012<=release_year<=2014')

In [34]:
data_find[data_find.profit == data_find.profit.min()]

,imdb_id,original_title,profit,release_year
1245,tt1210819,The Lone Ranger,-165710090,2013


In [35]:
answers['10'] = 'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [36]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
# Сначала рассмотрим случаи, если бы мы не производили предобработку данных
data_find = pd.read_csv('movie_bd_v5.csv')
genres_counter = Counter()  # создаем "счетчик жанров"

for genres in data_find.genres.str.split('|'):
    genres_counter += Counter(genres)

genres_counter.most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

In [37]:
answers['11'] = 'Drama'

ВАРИАНТ 2

In [38]:
genres_dict = {}  # создаем словарь жанров

for row in data_find.genres:
    for genres in row.split('|'):
        genres_dict.setdefault(genres, 0)
        genres_dict[genres] += 1

genres_dict

{'Action': 582,
 'Adventure': 415,
 'Science Fiction': 248,
 'Thriller': 596,
 'Fantasy': 222,
 'Crime': 315,
 'Western': 19,
 'Drama': 782,
 'Family': 260,
 'Animation': 139,
 'Comedy': 683,
 'Mystery': 168,
 'Romance': 308,
 'War': 58,
 'History': 62,
 'Music': 64,
 'Horror': 176,
 'Documentary': 8,
 'Foreign': 2}

In [39]:
answers['11'] = 'Drama'

ВАРИАНТ 3

In [40]:
data_find = data[['imdb_id', 'genres']].drop_duplicates()

In [41]:
data_find.genres.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

In [42]:
answers['11'] = 'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [43]:
data_find = data[['imdb_id', 'genres', 'profit']].drop_duplicates()
data_find = data_find.query('profit>0')

In [44]:
data_find['genres'].value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [45]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [46]:
data_find = data[['imdb_id', 'director', 'profit']].drop_duplicates()

In [47]:
# Сгруппируем данные по режиссеру
director_group = data_find.groupby(['director'])['profit'].sum()

In [48]:
director_group.sort_values(ascending=False)

director
Peter Jackson        5202593685
David Yates          3379295625
Christopher Nolan    3162548502
J.J. Abrams          2839169916
Michael Bay          2760938960
                        ...    
Jack Sholder          -75171919
Peter Hyams           -86956545
Ron Underwood         -92896027
James L. Brooks       -96289726
Walter Hill          -128283462
Name: profit, Length: 997, dtype: int64

In [49]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [50]:
data_find = data[['imdb_id', 'director', 'genres']].drop_duplicates()

In [51]:
# Создадим сводную таблицу 'режиссер-жанр'
pivot = data_find.pivot_table(
    values='imdb_id', index='director', columns='genres', aggfunc='count', fill_value=0)

In [52]:
pivot.Action.sort_values(ascending=False)

director
Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Mark Herman           0
Mark Helfrich         0
Mark Duplass          0
Mark Dindal           0
Jorge Blanco          0
Name: Action, Length: 997, dtype: int64

In [53]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [54]:
data_find = data[['imdb_id', 'cast', 'profit',
                  'release_year']].drop_duplicates()

In [55]:
# Выберим данные по условию (год выхода - 2012)
data_find = data_find[data_find.release_year == 2012]

In [56]:
# Сгрупируем данные по актеру
cast_group = data_find.groupby(['cast'])['profit'].sum()

In [57]:
cast_group.sort_values()

cast
Kirsten Dunst         -68109207
Heidi Hawkins         -51893525
James Kidnie          -51893525
Timothy Spall         -51893525
Malin Ã…kerman        -43733509
                        ...    
Robert Downey Jr.    1299557910
Mark Ruffalo         1299557910
Chris Evans          1299557910
Denis Leary          1319460639
Chris Hemsworth      1542450773
Name: profit, Length: 466, dtype: int64

In [58]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [59]:
data_find = data[['imdb_id', 'cast', 'budget']].drop_duplicates()

In [60]:
# Выберим данные по условию (бюджет выше среднего)
data_find = data_find[data_find.budget > data_find.budget.mean()]

In [61]:
# Сгрупируем данные по актеру
cast_group = data_find.groupby(['cast'])['imdb_id'].count()

In [62]:
cast_group.sort_values()

cast
50 Cent               1
Lena Olin             1
Lennie James          1
Leonard Nimoy         1
Leslie Bibb           1
                     ..
Samuel L. Jackson    15
Eddie Murphy         15
Angelina Jolie       16
Adam Sandler         17
Matt Damon           18
Name: imdb_id, Length: 1505, dtype: int64

In [63]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [64]:
data_find = data[['imdb_id', 'cast', 'genres']].drop_duplicates()

In [65]:
# Выберим данные по условию (снимался Nicolas Cage)
data_find = data_find[data_find.cast == 'Nicolas Cage']

In [66]:
data_find.genres.value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Animation           3
Family              3
Mystery             3
History             2
War                 1
Romance             1
Horror              1
Name: genres, dtype: int64

In [67]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [68]:
data_find = data[['imdb_id', 'original_title',
                  'production_companies', 'profit']].drop_duplicates()

In [69]:
# Выберим данные по условию (студия - Paramount Pictures)
data_find = data_find[data_find.production_companies == 'Paramount Pictures']

In [70]:
data_find[data_find.profit == data_find.profit.min()]

,imdb_id,original_title,production_companies,profit
925,tt0267626,K-19: The Widowmaker,Paramount Pictures,-64831034


In [71]:
answers['18'] = 'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [72]:
data_find = data[['imdb_id', 'revenue', 'release_year']].drop_duplicates()

In [73]:
# Сгруппируем данные в разрезе 'год выхода на экран' и сортируем результат
data_find.groupby(['release_year'])['revenue'].sum().sort_values()

release_year
2000    10664099805
2001    13017764865
2002    14136361487
2003    14346123312
2006    14775042320
2005    15309425558
2004    15663430720
2007    18162406801
2008    18252781990
2009    20261791024
2010    21071204961
2011    22676791872
2012    23079001687
2013    23213799791
2014    23405862953
2015    25449202382
Name: revenue, dtype: int64

In [74]:
answers['19'] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [75]:
data_find = data[['imdb_id', 'production_companies',
                  'release_year', 'profit']].drop_duplicates()

In [76]:
# Отфильтруем данные по студии - 'Warner Bros'
data_find[data_find.production_companies == 'Warner Bros']

,imdb_id,production_companies,release_year,profit


In [77]:
# Данных с таким названием студии не найдено, выполним поиск по наличию
# в названии строки 'arner'
product_companies = data_find[data_find.production_companies.str.contains(
    'arner')]

In [78]:
# Выведем уникальные названия
product_companies.production_companies.unique()

array(['Warner Bros.', 'Warner Bros. Animation',
       'Warner Bros. Interactive Entertainment', 'Warner Bros. Pictures',
       'Warner Independent Pictures (WIP)'], dtype=object)

In [79]:
# Есть 4 студии, соотвествующие условию задания, отфильтруем датафрейм по ним
warner_film = data_find[data_find.production_companies.str.contains(
    'Warner B')]

In [80]:
# сгруппируем данные (суммарная прибыль) по годам
year_group = warner_film.groupby(['release_year'])['profit'].sum()

In [81]:
# сортируем по годам
year_group.sort_values()

release_year
2000     452631386
2006     625351872
2015     870368348
2002    1022709901
2012    1258020056
2001    1282142579
2005    1551980298
2004    1631933725
2013    1636453400
2009    1822454136
2003    1844008221
2011    1871393682
2010    1974712985
2008    2134595031
2007    2201675217
2014    2292949646
Name: profit, dtype: int64

In [82]:
answers['20'] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [83]:
data_find = data[['imdb_id', 'release_month']].drop_duplicates()

In [84]:
# Группируем в разрезе 'месяц выхода на экран'
month_group = data_find.groupby(['release_month'])['imdb_id'].count()

In [85]:
# Сортируем значения
month_group.sort_values()

release_month
1     110
2     135
5     140
7     142
11    146
6     147
4     149
3     156
8     161
10    186
12    190
9     227
Name: imdb_id, dtype: int64

In [86]:
answers['21'] = '9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [87]:
# выполним запрос 'месяц выхода фильма с 6 до 8 включительно'
data_find = data_find.query('6<=release_month<=8')

In [88]:
# посчитаем количество фильмов в результате выполнения запроса
len(data_find)

450

In [89]:
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [90]:
data_find = data[['imdb_id', 'director', 'release_month']].drop_duplicates()

In [91]:
data_find = data_find.query('release_month in [1, 2, 12]')

In [92]:
# Сгруппируем и сортируем
direcor_group = data_find.groupby(['director'])['imdb_id'].count()
direcor_group.sort_values()

director
Jon Avnet                1
Matthew Vaughn           1
Matthew O'Callaghan      1
Matt Reeves              1
Matt Bettinelli-Olpin    1
                        ..
Adam Shankman            4
Martin Scorsese          4
Clint Eastwood           6
Steven Soderbergh        6
Peter Jackson            7
Name: imdb_id, Length: 358, dtype: int64

In [93]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [94]:
data_find = data[['imdb_id', 'production_companies',
                  'length_title']].drop_duplicates()

In [95]:
# Сгруппируем и сортируем
product_group = data_find.groupby(['production_companies'])[
    'length_title'].mean()

In [96]:
product_group.sort_values()

production_companies
Global Entertainment Group     2.0
Ixtlan Productions             2.0
XM2 Productions                2.0
Berlanti Productions           3.0
Everest Entertainment          3.0
                              ... 
Polsky Films                  46.0
Museum Canada Productions     46.0
Dos Corazones                 47.0
Jim Henson Company, The       59.0
Four By Two Productions       83.0
Name: length_title, Length: 1771, dtype: float64

In [97]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [98]:
data_find = data[['imdb_id', 'production_companies',
                  'count_word']].drop_duplicates()

In [99]:
product_group = data_find.groupby(['production_companies'])[
    'count_word'].mean()

In [100]:
product_group.sort_values()

production_companies
Motion Picture Corporation of America     11.0
Empire Pictures                           11.0
Henceforth                                13.0
Phantom Four                              13.0
London Boulevard                          13.0
                                         ...  
Brookwell-McNamara Entertainment         156.0
98 MPH Productions                       159.0
Heineken Branded Entertainment           161.0
Room 9 Entertainment                     161.0
Midnight Picture Show                    175.0
Name: count_word, Length: 1771, dtype: float64

In [101]:
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [102]:
data_find = data[['imdb_id', 'original_title',
                  'vote_average']].drop_duplicates()

In [103]:
# Определим количество фильмов, входящих в 1%
n = round(len(data_find)/100)

In [104]:
# Проведем сортировку по 'средней оценке'
sort_vote = data_find.sort_values('vote_average', ascending=False)

In [105]:
# Выведем перечень 1% лучших
sort_vote[0:n]

,imdb_id,original_title,vote_average
599,tt0468569,The Dark Knight,8.1
118,tt0816692,Interstellar,8.0
125,tt2084970,The Imitation Game,8.0
9,tt2096673,Inside Out,8.0
34,tt3170832,Room,8.0
1183,tt0993846,The Wolf of Wall Street,7.9
128,tt2267998,Gone Girl,7.9
1191,tt2024544,12 Years a Slave,7.9
119,tt2015381,Guardians of the Galaxy,7.9
1081,tt0167260,The Lord of the Rings: The Return of the King,7.9


In [106]:
answers['26'] = list(sort_vote[0:n]['original_title'])

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [107]:
# В данном задании с помощью метода combinations определим все пары актеров
# снимавшихся вместе в одном фильме, а с помощью метода Counter посчитаем количество
# всех таких комбинаций

In [108]:
# Для этого загрузим датафрейм в первоначальном виде (без разделения на отдельные ячейки)
# и превратим значения в колонке 'cast' в списки
data_find = pd.read_csv('movie_bd_v5.csv')
data_find.cast = data_find.cast.str.split('|')

In [109]:
cast_counter = Counter()
for casts in data_find.cast:
    path = list(combinations(casts, 2))
    cast_counter += Counter(path)

In [110]:
# наиболее частовстречающиеся пары
cast_counter.most_common(5)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

In [111]:
answers['27'] = list(cast_counter.most_common(2))

ВАРИАНТ 2

# Submission

In [112]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110',
 '5': '107',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  'Big Hero 6',
  'The Fault in Our Stars',
  'The Lord of the Rings: The Two Towers'],
 '27': [(('Da

In [113]:
# и убедиться что ни чего не пропустил)
len(answers)

27